In [27]:
import os, sys, re, json, time, unittest
import itertools, collections
import re
import operator
import numpy as np
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))
#nltk.download('maxent_ne_chunker')
#nltk.download('words')
#nltk.download('vader_lexicon')

In [ ]:
###############Method 1##################
#this is just a quick and dirty method
#Method 2 might be more useful
#positive and negative list came from a public source
#

In [2]:
#loading positive word list
pos_words = []
startbool = False
for line in open('positive-words.txt').readlines():
    if not line.startswith(";"):
        pos_words.append(line.strip('\r\n'))
pos_words = pos_words[1:]
print pos_words[:8]

['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable', 'accessible', 'acclaim']


In [3]:
#loading negative word list
neg_words = []
startbool = False
for line in open('negative-words.txt').readlines():
    if not line.startswith(";"):
        neg_words.append(line.strip('\r\n'))
neg_words = neg_words[1:]
print neg_words[:8]

['2-faced', '2-faces', 'abnormal', 'abolish', 'abominable', 'abominably', 'abominate', 'abomination']


In [9]:
lines = []
count = 0
for line in open('text_test_deps.csv').readlines():
    temp_str = line.strip('\r\n')
    if temp_str != "":
        sentences = nltk.sent_tokenize(temp_str.decode('ascii', 'ignore'))
        tokenized_sentences = [nltk.word_tokenize(sentence) for sentence in sentences]
        tagged_sentences = [nltk.pos_tag(sentence) for sentence in tokenized_sentences]
        lines.append(tagged_sentences)
#print len(lines)

1432787


In [ ]:
import csv

with open('lines_text_test_deps.csv', 'wb') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(lines)

In [116]:
print lines[1]

[[(u'0', 'CD'), (u',', ','), (u"''", "''"), (u'Need', 'VBP'), (u'a', 'DT'), (u'place', 'NN'), (u'to', 'TO'), (u'sleep', 'VB'), (u'that', 'DT'), (u'has', 'VBZ'), (u'easy', 'JJ'), (u'access', 'NN'), (u'to', 'TO'), (u'all', 'DT'), (u'of', 'IN'), (u'San', 'NNP'), (u'Francisco', 'NNP'), (u'?', '.')], [(u'Our', 'PRP$'), (u'living', 'NN'), (u'room', 'NN'), (u'sofabed', 'VBD'), (u'on', 'IN'), (u'the', 'DT'), (u'corner', 'NN'), (u'of', 'IN'), (u'Golden', 'NNP'), (u'Gate', 'NNP'), (u'Park', 'NNP'), (u'is', 'VBZ'), (u'in', 'IN'), (u'the', 'DT'), (u'center', 'NN'), (u'of', 'IN'), (u'the', 'DT'), (u'city', 'NN'), (u'!', '.')], [(u'As', 'IN'), (u'an', 'DT'), (u'added', 'JJ'), (u'bonus', 'NN'), (u',', ','), (u'access', 'NN'), (u'to', 'TO'), (u'public', 'JJ'), (u'transportation', 'NN'), (u'(', '('), (u'MUNI', 'NNP'), (u')', ')'), (u'is', 'VBZ'), (u'less', 'JJR'), (u'than', 'IN'), (u'a', 'DT'), (u'5', 'CD'), (u'minute', 'NN'), (u'walk', 'NN'), (u'away', 'RB'), (u'.', '.')], [(u'We', 'PRP'), (u"'ve", 'V

In [136]:
#only digested 8 lines to demonstrate. Will continue if needed.
scored_sentences = []
for review in lines[1:10]:
    score = 0
    temp_sentence = []
    for sentence in review:
        for word in sentence:
            if word[0].isalpha():
                if word[0] in pos_words:
                    score += 1
                if word[0] in neg_words:
                    score -= 1
                temp_sentence.append(word)

    scored_sentences.append([temp_sentence, score])
    
    
for line, score in scored_sentences:
    print line
    print score

[(u'Need', 'VBP'), (u'a', 'DT'), (u'place', 'NN'), (u'to', 'TO'), (u'sleep', 'VB'), (u'that', 'DT'), (u'has', 'VBZ'), (u'easy', 'JJ'), (u'access', 'NN'), (u'to', 'TO'), (u'all', 'DT'), (u'of', 'IN'), (u'San', 'NNP'), (u'Francisco', 'NNP'), (u'Our', 'PRP$'), (u'living', 'NN'), (u'room', 'NN'), (u'sofabed', 'VBD'), (u'on', 'IN'), (u'the', 'DT'), (u'corner', 'NN'), (u'of', 'IN'), (u'Golden', 'NNP'), (u'Gate', 'NNP'), (u'Park', 'NNP'), (u'is', 'VBZ'), (u'in', 'IN'), (u'the', 'DT'), (u'center', 'NN'), (u'of', 'IN'), (u'the', 'DT'), (u'city', 'NN'), (u'As', 'IN'), (u'an', 'DT'), (u'added', 'JJ'), (u'bonus', 'NN'), (u'access', 'NN'), (u'to', 'TO'), (u'public', 'JJ'), (u'transportation', 'NN'), (u'MUNI', 'NNP'), (u'is', 'VBZ'), (u'less', 'JJR'), (u'than', 'IN'), (u'a', 'DT'), (u'minute', 'NN'), (u'walk', 'NN'), (u'away', 'RB'), (u'We', 'PRP'), (u'converted', 'VBN'), (u'our', 'PRP$'), (u'living', 'NN'), (u'room', 'NN'), (u'sofa', 'NN'), (u'into', 'IN'), (u'a', 'DT'), (u'bed', 'NN'), (u'by', 'IN

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:11: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
##########Method 2##################

In [44]:
lines = []
count = 0
for line in open('text_test_deps.csv').readlines():
    temp_str = line.strip('\r\n')
    if temp_str != "":
        lines.append(temp_str)
print lines[1]

0,"Need a place to sleep that has easy access to all of San Francisco? Our living room sofabed on the corner of Golden Gate Park is in the center of the city! As an added bonus, access to public transportation (MUNI) is less than a 5 minute walk away. We've converted our living room sofa into a full-sized bed by outfitting our modular sofa with a traditional Japanese organic futon mattress. The bedrooms will be occupied by my roommate and myself. We are two young professional males in our mid-20s in the tech industry. Whether you're looking to see San Francisco highlights like the Golden Gate Bridge, Golden Gate Park, Mission Dolores Park, or Fisherman's Wharf, with this central location, you'll have easy access! There will be a lockbox located outside of the apartment building which means you will have your own set of keys so that you can conveniently go in and out of the unit at your discretion. You'll have full private access to the master bedroom. While the closet is in use, it's q

In [ ]:
#if you want a total sentiment.
#import nltk 
#import pandas as pd 
#import string
#messages = lines[:10]
#from nltk.sentiment.vader import SentimentIntensityAnalyzer

#sid = SentimentIntensityAnalyzer()
#summary = {"positive":0,"neutral":0,"negative":0}
#for x in messages: 
#    ss = sid.polarity_scores(x)
#    if ss["compound"] == 0.0: 
#        summary["neutral"] +=1
#    elif ss["compound"] > 0.0:
#        summary["positive"] +=1
#    else:
#        summary["negative"] +=1
#print summary

In [35]:
#this will use nltk's built in sentiment score
#more info on this: http://t-redactyl.io/blog/2017/04/using-vader-to-handle-sentiment-analysis-with-social-media-text.html
#only digested 8 lines to demonstrate. Will continue if needed.
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
sentiment = []
for each in lines[1:10]:
    ss = sid.polarity_scores(each)
    sentiment.append([each, ss])

In [135]:
#I believe we should focus on the compound score as that score is normalized to fit between -1 and 1
print sentiment[0]

['0,"Need a place to sleep that has easy access to all of San Francisco? Our living room sofabed on the corner of Golden Gate Park is in the center of the city! As an added bonus, access to public transportation (MUNI) is less than a 5 minute walk away. We\'ve converted our living room sofa into a full-sized bed by outfitting our modular sofa with a traditional Japanese organic futon mattress. The bedrooms will be occupied by my roommate and myself. We are two young professional males in our mid-20s in the tech industry. Whether you\'re looking to see San Francisco highlights like the Golden Gate Bridge, Golden Gate Park, Mission Dolores Park, or Fisherman\'s Wharf, with this central location, you\'ll have easy access! There will be a lockbox located outside of the apartment building which means you will have your own set of keys so that you can conveniently go in and out of the unit at your discretion. You\'ll have full private access to the master bedroom. While the closet is in use,